In [ ]:
# Only to mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Installing necesarry packages

! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] scikit-learn

In [ ]:
import os

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MarianMTModel, MarianTokenizer

from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import datasets

import random
import numpy as np
import pandas as pd
from IPython.display import display, HTML


os.environ["WANDB_DISABLED"]="true"
print(transformers.__version__)

4.16.2


In [ ]:
# All necessary variables
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zu"
batch_size = 8
epoch = 10
learning_rate=1e-5
weight_decay = 0.001
model_checkpoint = 'Helsinki-NLP/opus-mt-en-sn'# Pretrained Model path
model_name = model_checkpoint.split("/")[-1]

In [ ]:
# Uploading data
base_url = '/content/drive/MyDrive/Africa-NLP-Paper/'
raw_datasets = load_dataset('csv', data_files=[base_url + 'en-zu.training.csv'])
eval_dataset = load_dataset('csv', data_files=[base_url + 'en-zu.eval.csv'])
# Preparing data
eval_dataset = eval_dataset.remove_columns('zu.1')
eval_dataset = eval_dataset.rename_column('Source','source')
# Uploading Metrics
metric = load_metric("sacrebleu")


Using custom data configuration default-73c6d49c79903342
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-73c6d49c79903342/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-889bf343aa5166a9
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-889bf343aa5166a9/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Let's visualize a sentence
raw_datasets["train"][0]

{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.",
 'source': 'News Crawl 2019',
 'zu': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.'}

In [ ]:
# Let's visualize a sentence
eval_dataset["train"][0]

{'en': 'Peter Van Sant: And it means what?',
 'source': 'News Crawl 2019',
 'zu': 'Peter Van Sant: Bese kusho ukuthini?'}

In [ ]:

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,en,zu,source
0,Oct. 9 (UPI) -- Pop music star and television personality Gwen Stefani has been chosen to receive the E! People's Choice Awards Fashion Icon Award at a Nov. 10 ceremony in Santa Monica.,Ngo-Oct. 9 (UPI) – Isihlabani somculo wePop nomlingisi wethelevishini uGwen Stefani uye waqokwa ukuba athole u-E! People's Choice Awards Fashion Icon Award emcimbini wango-Nov. 10 eSanta Monica.,News Crawl 2019
1,"""9 Shockingly Tone-Deaf Ads That Should Have Never Happened""",Izikhangiso Ezithusayo Zezithoni Zabangezwa eziyi-9 Okwabe kungafanele Zenzeke ”,News Crawl 2019
2,The Commission's findings are very uncomfortable for Oxfam GB but we accept them.,Okutholwe yiKhomishini akuthokozisi kakhulu nge-Oxfam GB kepha siyakwamukela.\n,News Crawl 2019
3,That was - and still is - a world-leading goal that will be very hard to deliver.,Lokho bekuyikho - futhi kusenjalo - inhloso ehamba phambili emhlabeni okuzoba nzima kakhulu ukuyifeza.,News Crawl 2019
4,"That was enough to take the first set and the pattern continued, with more emphasis in a quickfire second.","Lokho kwakwanele ukuthatha isethi yokuqala futhi iphethini yaqhubeka, ngokugcizelela okwengeziwe ngomzuzwana osheshayo womlilo.",News Crawl 2019


In [ ]:

def preprocess_function(examples):
    '''
    Pre- and post-processing of tokens
    '''
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def postprocess_text(preds, labels):
    '''
    post processing the outputs for BLEU
    '''
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    '''
    computes BLEU
    '''
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
'''
This will select sentencpiece
'''    

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

In [ ]:
# Let's visualize some token embeddings
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[2314, 13282, 20571, 18, 4, 1270, 9, 128, 287, 7, 4, 411, 23857, 10, 40215, 3400, 13843, 47, 6, 517, 2309, 3409, 3, 0], [2079, 85, 61, 2842, 33, 235, 55, 639, 12, 5407, 4, 5010, 2, 55, 161, 45, 19411, 101, 963, 19233, 12, 399, 2210, 9835, 9387, 30, 648, 1094, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[84, 1973, 21, 253, 226, 3559, 1108, 15, 11230, 51, 484, 21, 9, 8002, 484, 2261, 133, 11152, 6236, 15901, 21, 2069, 49, 17463, 883, 10555, 15, 13770, 101, 7068, 484, 7896, 29222, 27028, 2635, 15225, 19549, 49, 26907, 9068, 8670, 81, 31708, 4932, 4642, 3, 0], [996, 13030, 6620, 2611, 8, 14270, 2635, 9807, 28085, 95, 13030, 1611, 51, 774, 8452, 13030, 24670, 49, 12310, 49, 7640, 93, 63, 49, 4096, 685, 24888, 5645, 18, 133, 186, 133, 2307, 2, 49, 9807, 28085, 8452, 14270, 4099, 51, 352, 774, 8452, 13030, 24670, 6566, 

In [ ]:
#Preparing training and validation data
tokenized_train_dataset = raw_datasets.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Getting pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

In [ ]:
'''
This is tuned to en-xh-sentencepiece model. I think comparing on 5 epochs is fine
'''
ckpt_path = f"{base_url}{model_name}-finetuned-{source_lang}-to-{target_lang}"
args = Seq2SeqTrainingArguments(
    ckpt_path,
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=3,
    num_train_epochs=epoch,
    predict_with_generate=True    
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # initializing data collator

In [ ]:
# Training args
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_eval_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: zu, source, en.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4739
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5930


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,4.571800,3.670168,4.098700,81.538100
2,3.785400,3.311732,3.417500,102.927900
3,3.511100,3.109020,3.483400,110.585200
4,3.329700,2.973727,2.955300,130.903800
5,3.188100,2.876923,3.148600,131.087200
6,3.008000,2.810040,3.278700,125.759500
7,2.946700,2.763809,4.971000,94.173300
8,2.901300,2.728142,5.134300,90.118200
9,2.868900,2.709389,5.299900,89.503000
10,2.852700,2.702641,5.426300,89.363700


Saving model checkpoint to /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500
Configuration saved in /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: zu, source, en.
***** Running Evaluation *****
  Num examples = 998
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-1000
Configuration saved

TrainOutput(global_step=5930, training_loss=3.2453165758721543, metrics={'train_runtime': 12542.159, 'train_samples_per_second': 3.778, 'train_steps_per_second': 0.473, 'total_flos': 814676226342912.0, 'train_loss': 3.2453165758721543, 'epoch': 10.0})

In [ ]:
# Testing our model against google translation
src_text = ['My name is Muhammad and I live in Africa']
model_name = f'{ckpt_path}/checkpoint-500'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]


Could not locate the tokenizer configuration file, will try to use the model config instead.
404 Client Error: Not Found for url: https://huggingface.co//content/drive/MyDrive/Africa-NLP-Paper/opus-mt-en-fr-finetuned-en-to-zu/checkpoint-500/resolve/main/config.json


OSError: ignored

In [ ]:
print("Google translation: \n Igama lami nginguMuhammad futhi ngihlala e-Afrika")